# Plotting 2-D data

Scipp uses the `imshow` function from the `matplotlib` library to visualize 2-D data.

In [ ]:
import numpy as np
import scipp as sc
from scipp.plot import plot

## Basic image plot

2-D variables are plotted as images, with a colormap:

In [ ]:
N = 100
M = 50
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
x, y = np.meshgrid(xx, yy)
b = N/20.0
c = M/2.0
r = np.sqrt(((x-c)/b)**2 + (y/b)**2)
a = np.sin(r)
d1 = sc.Dataset()
d1.coords['x'] = sc.Variable(['x'], values=xx, unit=sc.units.m)
d1.coords['y'] = sc.Variable(['y'], values=yy, unit=sc.units.m)
d1['Signal'] = sc.Variable(['y', 'x'], values=a, unit=sc.units.counts)
plot(d1)

The dimension displayed along each axis of the image can be selected with the `axes` keyword argument which accepts a list of dimensions:

In [ ]:
plot(d1, axes=['x', 'y'])

## Changing the colorscale

Changing the colorscale is handled in a similar way to the Matplotlib API. The colormap is defined by the `cmap` argument:

In [ ]:
plot(d1, cmap='magma')

A logarithmic colorscale is obtained by setting `log` to `True`:

In [ ]:
plot(d1, log=True)

Upper and lower limits on the colorscale can be placed using `vmin` and `vmax`:

In [ ]:
plot(d1, vmin=0, vmax=0.5)

## Using labels along some axis

Just like in the 1d plots, we can use labels along a chosen dimension:

In [ ]:
d1.coords['somelabels'] = sc.Variable(['x'],
                                      values=np.linspace(101., 155., N),
                                      unit=sc.units.s)
plot(d1, axes=['y', 'somelabels'])

## Collapsing dimensions

Sometimes it is useful to collapse one or more of the data's dimensions, if for instance most detector pixels contain noise, but one specific channel contains a strong signal. This is done by specifying the dimension to be displayed along the x axis as a keyword argument. All other dimensions will be collapsed.

In [ ]:
N = 40
M = 5
x = np.arange(N).astype(np.float64)
b = 0.5 * N
a = 4.0*np.random.rand(M, N)
a[2, :] = np.abs(10.0 * np.cos((x-b)*2.0/b))
d2 = sc.Dataset()
d2.coords['tof'] = sc.Variable(['tof'], values=x, unit=sc.units.us)
d2.coords['x'] = sc.Variable(['x'], values=np.arange(M).astype(np.float64),
                               unit=sc.units.m)
d2['sample'] = sc.Variable(['x', 'tof'], values=a,
                           variances=0.1*np.random.rand(M, N))
plot(d2)
plot(d2, collapse='tof')

## Image aspect ratio
By default, the aspect ratio of 2D images is not preserved; images are stretched to the size of the figure. You can choose to preserve the aspect ratio via the `aspect` keyword argument:

In [ ]:
plot(d2, aspect='equal')

You can also make this a permanent setting by editing the config file (possible options are `'equal'` and `'auto'`):

In [ ]:
sc.config.update({'plot.aspect': 'equal'})
sc.config.update({'plot.aspect': 'auto'})

## Plotting masks

If a dataset contains masks, they will appear as greyed out on the image:

In [ ]:
N = 100
M = 50
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
x, y = np.meshgrid(xx, yy)
b = N/20.0
c = M/2.0
r = np.sqrt(((x-c)/b)**2 + (y/b)**2)
a = np.sin(r)
d3 = sc.Dataset()
d3.coords['x'] = sc.Variable(['x'], values=xx, unit=sc.units.m)
d3.coords['y'] = sc.Variable(['y'], values=yy, unit=sc.units.m)
d3['Signal'] = sc.Variable(['y', 'x'], values=a, unit=sc.units.counts)
d3.masks['mask1'] = sc.Variable(['y', 'x'], values=np.where(a < 0, True, False))
plot(d3)

A toggle button below the plot can be used to hide/show the masks.

The mask can be represented as a solid color with

In [ ]:
plot(d3, masks={'color': 'magenta'})

We also note that any 1D mask will automatically broadcast onto a 2D image:

In [ ]:
d3.masks['mask1'] = sc.Variable(['x'], values=np.where(np.abs(xx-50) < 10, True, False))
plot(d3)

## Plotting event data
If a `Dataset` contains event data, it is possible to plot the data directly using some on-the-fly histogramming, the parameters of which are specified in the `bins` keyword argument. This can either be a single integer (the number of bins; in this case the extents of the events dimension are automatically computed), a numpy array (representing the bin edges), or a `scipp` Variable (representing the bin edges, see https://scipp.github.io/generated/scipp.histogram.html):

In [ ]:
N = 50
M = 10
var = sc.Variable(dims=['x'],
                  shape=[M],
                  dtype=sc.dtype.event_list_float64,
                  unit=sc.units.us)
dat = sc.Variable(dims=['x'],
                  values=np.ones(M),
                  variances=np.ones(M),
                  unit=sc.units.counts)
for i in range(M):
    v = np.random.normal(50.0, scale=20.0, size=int(np.random.rand()*N))
    var['x', i].values = v

d5 = sc.Dataset()
d5.coords['x'] = sc.Variable(['x'], values=np.arange(M), unit=sc.units.m)
d5['a'] = sc.DataArray(data=dat, coords={'tof': var})
sc.show(d5)
plot(d5, bins={'tof': 25})

## Saving figures
Static `pdf` or `png` copies of the figures can be saved to file (note that any buttons displayed under a figure are not saved to file). This is achieved as follows:

In [ ]:
plot(d3, filename='my_2d_figure.pdf')